# Other AWS Services: DynamoDB, Lambda, SQS, SNS

## Learning Objectives

By the end of this notebook, you will be able to:

1. Create and manage DynamoDB tables
2. Perform CRUD operations in DynamoDB
3. Invoke AWS Lambda functions
4. Work with SQS queues for messaging
5. Publish messages with SNS

---

## 1. DynamoDB Basics

### What is DynamoDB?

Amazon DynamoDB is a fully managed NoSQL database service that provides:
- **Single-digit millisecond** performance at any scale
- **Serverless** - no infrastructure management
- **Automatic scaling** based on demand
- **Built-in security** with encryption at rest

### Key Concepts

| Concept | Description | Example |
|---------|-------------|---------|
| **Table** | Collection of items | `Users`, `Orders` |
| **Item** | Single record | One user, one order |
| **Attribute** | Data field | `name`, `email`, `age` |
| **Primary Key** | Unique identifier | `user_id`, `order_id` |
| **Partition Key** | Determines data distribution | `user_id` |
| **Sort Key** | Optional secondary key | `created_at` |

### Primary Key Types

1. **Simple Primary Key** (Partition Key only)
   - Single attribute uniquely identifies each item
   - Example: `user_id`

2. **Composite Primary Key** (Partition Key + Sort Key)
   - Two attributes together uniquely identify each item
   - Example: `user_id` + `order_date`

In [ ]:
# Setup
import boto3
from botocore.exceptions import ClientError
import json
from decimal import Decimal
from typing import Dict, List, Any, Optional
from datetime import datetime

# Create DynamoDB client and resource
try:
    dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')
    dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
    print("DynamoDB client and resource created successfully")
except Exception as e:
    print(f"Could not create DynamoDB client: {e}")
    print("Continuing with code examples...")

### Creating Tables

In [ ]:
def create_table(
    table_name: str,
    partition_key: str,
    partition_key_type: str = 'S',
    sort_key: Optional[str] = None,
    sort_key_type: str = 'S'
) -> Dict[str, Any]:
    """
    Create a DynamoDB table.
    
    Args:
        table_name: Name of the table
        partition_key: Partition key attribute name
        partition_key_type: 'S' (String), 'N' (Number), or 'B' (Binary)
        sort_key: Optional sort key attribute name
        sort_key_type: Sort key type
        
    Returns:
        Dictionary with table information
    """
    dynamodb = boto3.resource('dynamodb')
    
    # Define key schema
    key_schema = [
        {'AttributeName': partition_key, 'KeyType': 'HASH'}  # Partition key
    ]
    
    attribute_definitions = [
        {'AttributeName': partition_key, 'AttributeType': partition_key_type}
    ]
    
    if sort_key:
        key_schema.append(
            {'AttributeName': sort_key, 'KeyType': 'RANGE'}  # Sort key
        )
        attribute_definitions.append(
            {'AttributeName': sort_key, 'AttributeType': sort_key_type}
        )
    
    try:
        table = dynamodb.create_table(
            TableName=table_name,
            KeySchema=key_schema,
            AttributeDefinitions=attribute_definitions,
            BillingMode='PAY_PER_REQUEST'  # On-demand pricing
        )
        
        # Wait for table to be created
        table.wait_until_exists()
        
        return {
            'success': True,
            'table_name': table_name,
            'status': table.table_status,
            'arn': table.table_arn
        }
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceInUseException':
            return {'success': False, 'error': 'Table already exists'}
        return {'success': False, 'error': str(e)}

print("Example: Create table")
print("-" * 40)
print("""
# Simple primary key (partition key only)
result = create_table(
    table_name='Users',
    partition_key='user_id',
    partition_key_type='S'
)

# Composite primary key (partition key + sort key)
result = create_table(
    table_name='UserOrders',
    partition_key='user_id',
    partition_key_type='S',
    sort_key='order_date',
    sort_key_type='S'
)
""")

### CRUD Operations

In [ ]:
class DynamoDBTable:
    """Helper class for DynamoDB table operations."""
    
    def __init__(self, table_name: str, region: str = 'us-east-1'):
        """
        Initialize table helper.
        
        Args:
            table_name: DynamoDB table name
            region: AWS region
        """
        self.dynamodb = boto3.resource('dynamodb', region_name=region)
        self.table = self.dynamodb.Table(table_name)
        self.table_name = table_name
    
    def put_item(self, item: Dict[str, Any]) -> bool:
        """
        Insert or replace an item.
        
        Args:
            item: Dictionary with item attributes
            
        Returns:
            True if successful
        """
        try:
            # Convert floats to Decimal (DynamoDB requirement)
            item = self._convert_floats(item)
            self.table.put_item(Item=item)
            return True
        except ClientError as e:
            print(f"Error putting item: {e}")
            return False
    
    def get_item(self, key: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        """
        Get a single item by key.
        
        Args:
            key: Dictionary with primary key attributes
            
        Returns:
            Item dictionary or None
        """
        try:
            response = self.table.get_item(Key=key)
            return response.get('Item')
        except ClientError as e:
            print(f"Error getting item: {e}")
            return None
    
    def update_item(
        self,
        key: Dict[str, Any],
        updates: Dict[str, Any]
    ) -> Optional[Dict[str, Any]]:
        """
        Update specific attributes of an item.
        
        Args:
            key: Primary key
            updates: Attributes to update
            
        Returns:
            Updated item or None
        """
        try:
            # Build update expression
            update_expression = "SET "
            expression_values = {}
            expression_names = {}
            
            for i, (attr, value) in enumerate(updates.items()):
                if i > 0:
                    update_expression += ", "
                
                # Use expression attribute names to handle reserved words
                attr_name = f"#attr{i}"
                value_name = f":val{i}"
                
                update_expression += f"{attr_name} = {value_name}"
                expression_names[attr_name] = attr
                expression_values[value_name] = self._convert_floats(value)
            
            response = self.table.update_item(
                Key=key,
                UpdateExpression=update_expression,
                ExpressionAttributeNames=expression_names,
                ExpressionAttributeValues=expression_values,
                ReturnValues='ALL_NEW'
            )
            
            return response.get('Attributes')
            
        except ClientError as e:
            print(f"Error updating item: {e}")
            return None
    
    def delete_item(self, key: Dict[str, Any]) -> bool:
        """
        Delete an item.
        
        Args:
            key: Primary key of item to delete
            
        Returns:
            True if successful
        """
        try:
            self.table.delete_item(Key=key)
            return True
        except ClientError as e:
            print(f"Error deleting item: {e}")
            return False
    
    def query(
        self,
        partition_key_name: str,
        partition_key_value: Any,
        sort_key_condition: Optional[Dict[str, Any]] = None
    ) -> List[Dict[str, Any]]:
        """
        Query items by partition key.
        
        Args:
            partition_key_name: Name of partition key
            partition_key_value: Value to query for
            sort_key_condition: Optional sort key condition
            
        Returns:
            List of matching items
        """
        from boto3.dynamodb.conditions import Key
        
        try:
            key_condition = Key(partition_key_name).eq(partition_key_value)
            
            if sort_key_condition:
                sk_name = sort_key_condition['name']
                sk_value = sort_key_condition['value']
                sk_operator = sort_key_condition.get('operator', 'eq')
                
                sk = Key(sk_name)
                if sk_operator == 'eq':
                    key_condition = key_condition & sk.eq(sk_value)
                elif sk_operator == 'begins_with':
                    key_condition = key_condition & sk.begins_with(sk_value)
                elif sk_operator == 'between':
                    key_condition = key_condition & sk.between(
                        sk_value[0], sk_value[1]
                    )
            
            response = self.table.query(KeyConditionExpression=key_condition)
            return response.get('Items', [])
            
        except ClientError as e:
            print(f"Error querying: {e}")
            return []
    
    def scan(self, filter_expression=None) -> List[Dict[str, Any]]:
        """
        Scan entire table (use sparingly!).
        
        Args:
            filter_expression: Optional filter
            
        Returns:
            List of items
        """
        try:
            if filter_expression:
                response = self.table.scan(FilterExpression=filter_expression)
            else:
                response = self.table.scan()
            return response.get('Items', [])
        except ClientError as e:
            print(f"Error scanning: {e}")
            return []
    
    def _convert_floats(self, obj):
        """Convert floats to Decimal for DynamoDB."""
        if isinstance(obj, float):
            return Decimal(str(obj))
        elif isinstance(obj, dict):
            return {k: self._convert_floats(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [self._convert_floats(item) for item in obj]
        return obj

print("DynamoDB CRUD operations class created")

In [ ]:
# Example usage of DynamoDBTable
print("Example: CRUD Operations")
print("-" * 40)
print("""
# Initialize table helper
users = DynamoDBTable('Users')

# CREATE - Insert a new user
users.put_item({
    'user_id': 'user_001',
    'name': 'Alice Smith',
    'email': 'alice@example.com',
    'age': 30,
    'created_at': '2024-01-15T10:00:00Z'
})

# READ - Get user by ID
user = users.get_item({'user_id': 'user_001'})
print(user)
# Output: {'user_id': 'user_001', 'name': 'Alice Smith', ...}

# UPDATE - Update user attributes
updated = users.update_item(
    key={'user_id': 'user_001'},
    updates={'age': 31, 'email': 'alice.smith@example.com'}
)
print(updated)

# DELETE - Remove user
users.delete_item({'user_id': 'user_001'})
""")

In [ ]:
# Query examples
print("Example: Query Operations")
print("-" * 40)
print("""
# Table with composite key: user_id (PK) + order_date (SK)
orders = DynamoDBTable('UserOrders')

# Query all orders for a user
user_orders = orders.query(
    partition_key_name='user_id',
    partition_key_value='user_001'
)

# Query orders for a specific date range
recent_orders = orders.query(
    partition_key_name='user_id',
    partition_key_value='user_001',
    sort_key_condition={
        'name': 'order_date',
        'operator': 'between',
        'value': ['2024-01-01', '2024-03-31']
    }
)

# Query orders starting with specific prefix
jan_orders = orders.query(
    partition_key_name='user_id',
    partition_key_value='user_001',
    sort_key_condition={
        'name': 'order_date',
        'operator': 'begins_with',
        'value': '2024-01'
    }
)
""")

### Batch Operations

In [ ]:
def batch_write_items(
    table_name: str,
    items: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Write multiple items in batches.
    
    Args:
        table_name: DynamoDB table name
        items: List of items to write
        
    Returns:
        Results dictionary
    """
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(table_name)
    
    result = {'written': 0, 'failed': 0}
    
    # DynamoDB batch_write_item has a limit of 25 items
    batch_size = 25
    
    with table.batch_writer() as batch:
        for item in items:
            try:
                batch.put_item(Item=item)
                result['written'] += 1
            except Exception as e:
                result['failed'] += 1
                print(f"Failed to write item: {e}")
    
    return result

print("Example: Batch write")
print("-" * 40)
print("""
users_data = [
    {'user_id': 'user_001', 'name': 'Alice', 'email': 'alice@example.com'},
    {'user_id': 'user_002', 'name': 'Bob', 'email': 'bob@example.com'},
    {'user_id': 'user_003', 'name': 'Charlie', 'email': 'charlie@example.com'},
]

result = batch_write_items('Users', users_data)
print(f"Written: {result['written']}, Failed: {result['failed']}")
""")

## 2. AWS Lambda

### What is AWS Lambda?

AWS Lambda lets you run code without provisioning servers:
- **Event-driven** - triggered by AWS services or HTTP
- **Auto-scaling** - handles any number of requests
- **Pay-per-use** - charged only for compute time
- Supports Python, Node.js, Java, Go, .NET, Ruby

In [ ]:
# Create Lambda client
lambda_client = boto3.client('lambda', region_name='us-east-1')
print("Lambda client created")

In [ ]:
def invoke_lambda(
    function_name: str,
    payload: Dict[str, Any],
    invocation_type: str = 'RequestResponse'
) -> Dict[str, Any]:
    """
    Invoke a Lambda function.
    
    Args:
        function_name: Name or ARN of the Lambda function
        payload: Input data for the function
        invocation_type: 
            'RequestResponse' - synchronous (wait for response)
            'Event' - asynchronous (fire and forget)
            'DryRun' - validate without executing
            
    Returns:
        Dictionary with invocation result
    """
    lambda_client = boto3.client('lambda')
    
    try:
        response = lambda_client.invoke(
            FunctionName=function_name,
            InvocationType=invocation_type,
            Payload=json.dumps(payload)
        )
        
        result = {
            'status_code': response['StatusCode'],
            'executed_version': response.get('ExecutedVersion'),
            'function_error': response.get('FunctionError')
        }
        
        # Read response payload for synchronous invocation
        if invocation_type == 'RequestResponse':
            response_payload = response['Payload'].read().decode('utf-8')
            result['payload'] = json.loads(response_payload) if response_payload else None
        
        return result
        
    except ClientError as e:
        return {'error': str(e)}

print("Example: Invoke Lambda")
print("-" * 40)
print("""
# Synchronous invocation
result = invoke_lambda(
    function_name='my-function',
    payload={'name': 'Alice', 'action': 'greet'}
)
print(result['payload'])  # Function's return value

# Asynchronous invocation (fire and forget)
result = invoke_lambda(
    function_name='my-async-function',
    payload={'task': 'process_data'},
    invocation_type='Event'
)
# status_code: 202 (accepted)
""")

In [ ]:
def list_lambda_functions(region: str = 'us-east-1') -> List[Dict[str, Any]]:
    """
    List all Lambda functions in a region.
    
    Returns:
        List of function information
    """
    lambda_client = boto3.client('lambda', region_name=region)
    
    functions = []
    paginator = lambda_client.get_paginator('list_functions')
    
    for page in paginator.paginate():
        for func in page['Functions']:
            functions.append({
                'name': func['FunctionName'],
                'runtime': func.get('Runtime', 'N/A'),
                'memory': func['MemorySize'],
                'timeout': func['Timeout'],
                'last_modified': func['LastModified']
            })
    
    return functions

# Example of Lambda function code structure
print("Example Lambda Function Code (Python):")
print("-" * 40)
lambda_code = '''
import json

def lambda_handler(event, context):
    """
    Lambda handler function.
    
    Args:
        event: Input event data (dict)
        context: Lambda context object
        
    Returns:
        Response dictionary
    """
    name = event.get('name', 'World')
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': f'Hello, {name}!',
            'input': event
        })
    }
'''
print(lambda_code)

## 3. Amazon SQS (Simple Queue Service)

### What is SQS?

Amazon SQS is a fully managed message queuing service:
- **Decouple** application components
- **Scale** independently
- **Reliable** message delivery
- Two queue types: Standard and FIFO

In [ ]:
# Create SQS client
sqs = boto3.client('sqs', region_name='us-east-1')
print("SQS client created")

In [ ]:
class SQSQueue:
    """Helper class for SQS operations."""
    
    def __init__(self, queue_url: str = None, queue_name: str = None):
        """
        Initialize SQS helper.
        
        Args:
            queue_url: Queue URL (if known)
            queue_name: Queue name (will lookup URL)
        """
        self.sqs = boto3.client('sqs')
        
        if queue_url:
            self.queue_url = queue_url
        elif queue_name:
            response = self.sqs.get_queue_url(QueueName=queue_name)
            self.queue_url = response['QueueUrl']
        else:
            raise ValueError("Either queue_url or queue_name required")
    
    def send_message(self, message_body: str, attributes: Dict = None) -> str:
        """
        Send a message to the queue.
        
        Args:
            message_body: Message content
            attributes: Optional message attributes
            
        Returns:
            Message ID
        """
        params = {
            'QueueUrl': self.queue_url,
            'MessageBody': message_body
        }
        
        if attributes:
            params['MessageAttributes'] = {
                key: {
                    'DataType': 'String',
                    'StringValue': str(value)
                }
                for key, value in attributes.items()
            }
        
        response = self.sqs.send_message(**params)
        return response['MessageId']
    
    def receive_messages(
        self,
        max_messages: int = 10,
        wait_time: int = 20,
        visibility_timeout: int = 30
    ) -> List[Dict[str, Any]]:
        """
        Receive messages from the queue.
        
        Args:
            max_messages: Maximum messages to receive (1-10)
            wait_time: Long polling wait time in seconds
            visibility_timeout: Time message is hidden after receipt
            
        Returns:
            List of messages
        """
        response = self.sqs.receive_message(
            QueueUrl=self.queue_url,
            MaxNumberOfMessages=min(max_messages, 10),
            WaitTimeSeconds=wait_time,
            VisibilityTimeout=visibility_timeout,
            MessageAttributeNames=['All']
        )
        
        messages = []
        for msg in response.get('Messages', []):
            messages.append({
                'message_id': msg['MessageId'],
                'receipt_handle': msg['ReceiptHandle'],
                'body': msg['Body'],
                'attributes': msg.get('MessageAttributes', {})
            })
        
        return messages
    
    def delete_message(self, receipt_handle: str) -> bool:
        """
        Delete a message after processing.
        
        Args:
            receipt_handle: Receipt handle from receive_message
            
        Returns:
            True if successful
        """
        try:
            self.sqs.delete_message(
                QueueUrl=self.queue_url,
                ReceiptHandle=receipt_handle
            )
            return True
        except ClientError:
            return False
    
    def get_queue_attributes(self) -> Dict[str, Any]:
        """Get queue attributes."""
        response = self.sqs.get_queue_attributes(
            QueueUrl=self.queue_url,
            AttributeNames=['All']
        )
        return response['Attributes']

print("SQS Queue helper class created")

In [ ]:
print("Example: SQS Operations")
print("-" * 40)
print("""
# Initialize queue
queue = SQSQueue(queue_name='my-queue')

# Send a message
message_id = queue.send_message(
    message_body=json.dumps({'task': 'process_order', 'order_id': '12345'}),
    attributes={'priority': 'high', 'source': 'web'}
)
print(f"Sent message: {message_id}")

# Receive and process messages
messages = queue.receive_messages(max_messages=5, wait_time=10)

for msg in messages:
    print(f"Processing: {msg['body']}")
    
    # Process the message...
    data = json.loads(msg['body'])
    
    # Delete after successful processing
    queue.delete_message(msg['receipt_handle'])
    print(f"Deleted message: {msg['message_id']}")
""")

In [ ]:
def create_sqs_queue(
    queue_name: str,
    fifo: bool = False,
    visibility_timeout: int = 30,
    message_retention: int = 345600  # 4 days
) -> str:
    """
    Create an SQS queue.
    
    Args:
        queue_name: Name of the queue
        fifo: Create FIFO queue (name must end with .fifo)
        visibility_timeout: Default visibility timeout
        message_retention: Message retention period in seconds
        
    Returns:
        Queue URL
    """
    sqs = boto3.client('sqs')
    
    attributes = {
        'VisibilityTimeout': str(visibility_timeout),
        'MessageRetentionPeriod': str(message_retention)
    }
    
    if fifo:
        if not queue_name.endswith('.fifo'):
            queue_name += '.fifo'
        attributes['FifoQueue'] = 'true'
        attributes['ContentBasedDeduplication'] = 'true'
    
    response = sqs.create_queue(
        QueueName=queue_name,
        Attributes=attributes
    )
    
    return response['QueueUrl']

print("Queue types:")
print("-" * 40)
print("""
STANDARD Queue:
- Best-effort ordering
- At-least-once delivery
- Nearly unlimited throughput

FIFO Queue:
- Exactly-once processing
- Strict ordering
- 300 messages/second (3000 with batching)
- Name must end with .fifo
""")

## 4. Amazon SNS (Simple Notification Service)

### What is SNS?

Amazon SNS is a pub/sub messaging service:
- **Topics** - channels for message delivery
- **Subscriptions** - endpoints that receive messages
- Multiple protocols: HTTP, Email, SMS, SQS, Lambda

In [ ]:
class SNSTopic:
    """Helper class for SNS operations."""
    
    def __init__(self, topic_arn: str = None, topic_name: str = None):
        """
        Initialize SNS helper.
        
        Args:
            topic_arn: Topic ARN (if known)
            topic_name: Topic name (will create if doesn't exist)
        """
        self.sns = boto3.client('sns')
        
        if topic_arn:
            self.topic_arn = topic_arn
        elif topic_name:
            response = self.sns.create_topic(Name=topic_name)
            self.topic_arn = response['TopicArn']
        else:
            raise ValueError("Either topic_arn or topic_name required")
    
    def publish(
        self,
        message: str,
        subject: str = None,
        attributes: Dict = None
    ) -> str:
        """
        Publish a message to the topic.
        
        Args:
            message: Message body
            subject: Optional subject (for email)
            attributes: Optional message attributes
            
        Returns:
            Message ID
        """
        params = {
            'TopicArn': self.topic_arn,
            'Message': message
        }
        
        if subject:
            params['Subject'] = subject
        
        if attributes:
            params['MessageAttributes'] = {
                key: {
                    'DataType': 'String',
                    'StringValue': str(value)
                }
                for key, value in attributes.items()
            }
        
        response = self.sns.publish(**params)
        return response['MessageId']
    
    def subscribe_email(self, email: str) -> str:
        """Subscribe an email address."""
        response = self.sns.subscribe(
            TopicArn=self.topic_arn,
            Protocol='email',
            Endpoint=email
        )
        return response['SubscriptionArn']
    
    def subscribe_sqs(self, queue_arn: str) -> str:
        """Subscribe an SQS queue."""
        response = self.sns.subscribe(
            TopicArn=self.topic_arn,
            Protocol='sqs',
            Endpoint=queue_arn
        )
        return response['SubscriptionArn']
    
    def subscribe_lambda(self, function_arn: str) -> str:
        """Subscribe a Lambda function."""
        response = self.sns.subscribe(
            TopicArn=self.topic_arn,
            Protocol='lambda',
            Endpoint=function_arn
        )
        return response['SubscriptionArn']
    
    def list_subscriptions(self) -> List[Dict[str, str]]:
        """List all subscriptions to this topic."""
        subscriptions = []
        paginator = self.sns.get_paginator('list_subscriptions_by_topic')
        
        for page in paginator.paginate(TopicArn=self.topic_arn):
            for sub in page['Subscriptions']:
                subscriptions.append({
                    'arn': sub['SubscriptionArn'],
                    'protocol': sub['Protocol'],
                    'endpoint': sub['Endpoint']
                })
        
        return subscriptions

print("SNS Topic helper class created")

In [ ]:
print("Example: SNS Operations")
print("-" * 40)
print("""
# Create or get topic
notifications = SNSTopic(topic_name='order-notifications')

# Subscribe endpoints
notifications.subscribe_email('admin@example.com')
notifications.subscribe_sqs('arn:aws:sqs:us-east-1:123456789:order-queue')

# Publish a notification
message_id = notifications.publish(
    message=json.dumps({
        'event': 'order_placed',
        'order_id': '12345',
        'amount': 99.99
    }),
    subject='New Order Received',
    attributes={'priority': 'high'}
)
print(f"Published message: {message_id}")

# List subscriptions
subs = notifications.list_subscriptions()
for sub in subs:
    print(f"  {sub['protocol']}: {sub['endpoint']}")
""")

### SNS + SQS Fan-Out Pattern

In [ ]:
print("SNS Fan-Out Pattern")
print("-" * 40)
print("""
This pattern allows one message to be delivered to multiple SQS queues:

                    +---> SQS Queue 1 (Order Processing)
                    |
Publisher --> SNS Topic ---> SQS Queue 2 (Inventory Update)
                    |
                    +---> SQS Queue 3 (Email Notifications)
                    |
                    +---> Lambda Function (Analytics)

Benefits:
- Decouple publishers from subscribers
- Add/remove subscribers without changing publisher
- Each queue processes independently
- Built-in retry and dead-letter support
""")

---

## Exercises

### Exercise 1: DynamoDB User Repository

Create a UserRepository class that manages users in DynamoDB with proper validation.

In [ ]:
# Exercise 1: Your code here
from dataclasses import dataclass
from typing import Optional, List
import re

@dataclass
class User:
    user_id: str
    email: str
    name: str
    age: Optional[int] = None
    created_at: Optional[str] = None

class UserRepository:
    """Repository for user operations in DynamoDB."""
    
    def __init__(self, table_name: str = 'Users'):
        pass
    
    def create(self, user: User) -> bool:
        """Create a new user with validation."""
        pass
    
    def get(self, user_id: str) -> Optional[User]:
        """Get user by ID."""
        pass
    
    def update(self, user_id: str, **kwargs) -> Optional[User]:
        """Update user attributes."""
        pass
    
    def delete(self, user_id: str) -> bool:
        """Delete a user."""
        pass
    
    def find_by_email(self, email: str) -> Optional[User]:
        """Find user by email (requires GSI)."""
        pass

<details>
<summary>Click to see solution</summary>

```python
import boto3
from botocore.exceptions import ClientError
from dataclasses import dataclass, asdict
from typing import Optional, List
from datetime import datetime
import re
from decimal import Decimal

@dataclass
class User:
    user_id: str
    email: str
    name: str
    age: Optional[int] = None
    created_at: Optional[str] = None

class UserRepository:
    """Repository for user operations in DynamoDB."""
    
    EMAIL_REGEX = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
    
    def __init__(self, table_name: str = 'Users'):
        """Initialize repository."""
        self.dynamodb = boto3.resource('dynamodb')
        self.table = self.dynamodb.Table(table_name)
        self.table_name = table_name
    
    def _validate_email(self, email: str) -> bool:
        """Validate email format."""
        return bool(self.EMAIL_REGEX.match(email))
    
    def _validate_user(self, user: User) -> List[str]:
        """Validate user data."""
        errors = []
        
        if not user.user_id or len(user.user_id) < 3:
            errors.append("user_id must be at least 3 characters")
        
        if not self._validate_email(user.email):
            errors.append("Invalid email format")
        
        if not user.name or len(user.name) < 1:
            errors.append("Name is required")
        
        if user.age is not None and (user.age < 0 or user.age > 150):
            errors.append("Age must be between 0 and 150")
        
        return errors
    
    def create(self, user: User) -> bool:
        """
        Create a new user with validation.
        
        Args:
            user: User object to create
            
        Returns:
            True if successful
            
        Raises:
            ValueError: If validation fails
        """
        errors = self._validate_user(user)
        if errors:
            raise ValueError(f"Validation errors: {', '.join(errors)}")
        
        # Set created_at if not provided
        if not user.created_at:
            user.created_at = datetime.utcnow().isoformat()
        
        try:
            # Check if user already exists
            self.table.put_item(
                Item=asdict(user),
                ConditionExpression='attribute_not_exists(user_id)'
            )
            return True
        except ClientError as e:
            if e.response['Error']['Code'] == 'ConditionalCheckFailedException':
                raise ValueError(f"User {user.user_id} already exists")
            raise
    
    def get(self, user_id: str) -> Optional[User]:
        """
        Get user by ID.
        
        Args:
            user_id: User ID to look up
            
        Returns:
            User object or None
        """
        try:
            response = self.table.get_item(Key={'user_id': user_id})
            item = response.get('Item')
            if item:
                return User(**item)
            return None
        except ClientError:
            return None
    
    def update(self, user_id: str, **kwargs) -> Optional[User]:
        """
        Update user attributes.
        
        Args:
            user_id: User ID to update
            **kwargs: Attributes to update
            
        Returns:
            Updated User object or None
        """
        # Validate email if being updated
        if 'email' in kwargs and not self._validate_email(kwargs['email']):
            raise ValueError("Invalid email format")
        
        # Build update expression
        update_expr = "SET "
        expr_values = {}
        expr_names = {}
        
        for i, (key, value) in enumerate(kwargs.items()):
            if key == 'user_id':  # Can't update primary key
                continue
            if i > 0:
                update_expr += ", "
            update_expr += f"#attr{i} = :val{i}"
            expr_names[f"#attr{i}"] = key
            expr_values[f":val{i}"] = value
        
        try:
            response = self.table.update_item(
                Key={'user_id': user_id},
                UpdateExpression=update_expr,
                ExpressionAttributeNames=expr_names,
                ExpressionAttributeValues=expr_values,
                ReturnValues='ALL_NEW',
                ConditionExpression='attribute_exists(user_id)'
            )
            return User(**response['Attributes'])
        except ClientError as e:
            if e.response['Error']['Code'] == 'ConditionalCheckFailedException':
                return None
            raise
    
    def delete(self, user_id: str) -> bool:
        """
        Delete a user.
        
        Args:
            user_id: User ID to delete
            
        Returns:
            True if deleted, False if not found
        """
        try:
            self.table.delete_item(
                Key={'user_id': user_id},
                ConditionExpression='attribute_exists(user_id)'
            )
            return True
        except ClientError as e:
            if e.response['Error']['Code'] == 'ConditionalCheckFailedException':
                return False
            raise
    
    def find_by_email(self, email: str) -> Optional[User]:
        """
        Find user by email.
        Note: This requires a Global Secondary Index on email.
        Without GSI, this does a full table scan (expensive!).
        """
        from boto3.dynamodb.conditions import Attr
        
        response = self.table.scan(
            FilterExpression=Attr('email').eq(email)
        )
        
        items = response.get('Items', [])
        if items:
            return User(**items[0])
        return None

# Example usage
repo = UserRepository('Users')

# Create user
user = User(
    user_id='user_001',
    email='alice@example.com',
    name='Alice Smith',
    age=30
)
# repo.create(user)

# Get user
# user = repo.get('user_001')

# Update user
# updated = repo.update('user_001', age=31, name='Alice Johnson')

# Delete user
# repo.delete('user_001')
```
</details>

### Exercise 2: Message Queue Processor

Create a message processor that reads from SQS, processes messages, and handles errors.

In [ ]:
# Exercise 2: Your code here
from typing import Callable
import time

class MessageProcessor:
    """Process messages from SQS queue."""
    
    def __init__(self, queue_name: str):
        pass
    
    def process_messages(
        self,
        handler: Callable[[dict], bool],
        max_messages: int = 10,
        poll_interval: int = 20
    ) -> dict:
        """Process messages with the given handler."""
        pass
    
    def run_forever(self, handler: Callable[[dict], bool]):
        """Continuously process messages."""
        pass

<details>
<summary>Click to see solution</summary>

```python
import boto3
import json
import time
from typing import Callable, Dict, Any
from botocore.exceptions import ClientError
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class MessageProcessor:
    """Process messages from SQS queue with error handling."""
    
    def __init__(self, queue_name: str, region: str = 'us-east-1'):
        """
        Initialize processor.
        
        Args:
            queue_name: SQS queue name
            region: AWS region
        """
        self.sqs = boto3.client('sqs', region_name=region)
        response = self.sqs.get_queue_url(QueueName=queue_name)
        self.queue_url = response['QueueUrl']
        self.queue_name = queue_name
        self._running = False
    
    def process_messages(
        self,
        handler: Callable[[dict], bool],
        max_messages: int = 10,
        poll_interval: int = 20,
        visibility_timeout: int = 30
    ) -> Dict[str, Any]:
        """
        Process a batch of messages.
        
        Args:
            handler: Function that processes message and returns True on success
            max_messages: Maximum messages to receive (1-10)
            poll_interval: Long polling wait time
            visibility_timeout: Time before message becomes visible again
            
        Returns:
            Results dictionary
        """
        results = {
            'received': 0,
            'processed': 0,
            'failed': 0,
            'errors': []
        }
        
        try:
            response = self.sqs.receive_message(
                QueueUrl=self.queue_url,
                MaxNumberOfMessages=min(max_messages, 10),
                WaitTimeSeconds=poll_interval,
                VisibilityTimeout=visibility_timeout,
                MessageAttributeNames=['All']
            )
        except ClientError as e:
            logger.error(f"Error receiving messages: {e}")
            results['errors'].append(str(e))
            return results
        
        messages = response.get('Messages', [])
        results['received'] = len(messages)
        
        for message in messages:
            message_id = message['MessageId']
            receipt_handle = message['ReceiptHandle']
            
            try:
                # Parse message body
                body = message['Body']
                try:
                    data = json.loads(body)
                except json.JSONDecodeError:
                    data = {'raw': body}
                
                # Call handler
                logger.info(f"Processing message {message_id}")
                success = handler(data)
                
                if success:
                    # Delete message on success
                    self.sqs.delete_message(
                        QueueUrl=self.queue_url,
                        ReceiptHandle=receipt_handle
                    )
                    results['processed'] += 1
                    logger.info(f"Successfully processed {message_id}")
                else:
                    results['failed'] += 1
                    logger.warning(f"Handler returned False for {message_id}")
                    
            except Exception as e:
                results['failed'] += 1
                results['errors'].append({
                    'message_id': message_id,
                    'error': str(e)
                })
                logger.error(f"Error processing {message_id}: {e}")
        
        return results
    
    def run_forever(
        self,
        handler: Callable[[dict], bool],
        error_sleep: int = 5
    ):
        """
        Continuously process messages until stopped.
        
        Args:
            handler: Message handler function
            error_sleep: Sleep time on error
        """
        self._running = True
        logger.info(f"Starting message processor for {self.queue_name}")
        
        while self._running:
            try:
                results = self.process_messages(handler)
                
                if results['errors']:
                    logger.warning(f"Batch had {len(results['errors'])} errors")
                    
            except Exception as e:
                logger.error(f"Critical error: {e}")
                time.sleep(error_sleep)
        
        logger.info("Message processor stopped")
    
    def stop(self):
        """Stop the processor."""
        self._running = False

# Example handler
def order_handler(message: dict) -> bool:
    """Process an order message."""
    order_id = message.get('order_id')
    if not order_id:
        return False
    
    print(f"Processing order: {order_id}")
    # Do actual processing here...
    return True

# Usage
# processor = MessageProcessor('order-queue')
# processor.run_forever(order_handler)
```
</details>

### Exercise 3: Event-Driven Architecture Simulator

Simulate an event-driven architecture with SNS topics and SQS queues.

In [ ]:
# Exercise 3: Your code here

class EventBus:
    """Simple event bus using SNS/SQS."""
    
    def __init__(self, name: str):
        pass
    
    def publish(self, event_type: str, data: dict):
        """Publish an event."""
        pass
    
    def subscribe(self, event_type: str, handler: Callable):
        """Subscribe to an event type."""
        pass

<details>
<summary>Click to see solution</summary>

```python
import boto3
import json
from typing import Callable, Dict, List, Any
from datetime import datetime
import uuid

class EventBus:
    """
    Simple event bus using SNS for publishing and SQS for subscribing.
    Demonstrates event-driven architecture patterns.
    """
    
    def __init__(self, name: str, region: str = 'us-east-1'):
        """
        Initialize event bus.
        
        Args:
            name: Name prefix for SNS topic and SQS queues
            region: AWS region
        """
        self.name = name
        self.region = region
        self.sns = boto3.client('sns', region_name=region)
        self.sqs = boto3.client('sqs', region_name=region)
        
        # Create main topic
        response = self.sns.create_topic(Name=f"{name}-events")
        self.topic_arn = response['TopicArn']
        
        # Track subscriptions
        self._subscriptions: Dict[str, List[Dict]] = {}
    
    def publish(self, event_type: str, data: dict) -> str:
        """
        Publish an event to the bus.
        
        Args:
            event_type: Type of event (e.g., 'order.created')
            data: Event payload
            
        Returns:
            Message ID
        """
        event = {
            'event_id': str(uuid.uuid4()),
            'event_type': event_type,
            'timestamp': datetime.utcnow().isoformat(),
            'data': data
        }
        
        response = self.sns.publish(
            TopicArn=self.topic_arn,
            Message=json.dumps(event),
            MessageAttributes={
                'event_type': {
                    'DataType': 'String',
                    'StringValue': event_type
                }
            }
        )
        
        return response['MessageId']
    
    def subscribe(
        self,
        subscriber_name: str,
        event_types: List[str] = None
    ) -> Dict[str, str]:
        """
        Create a subscription (SQS queue) for events.
        
        Args:
            subscriber_name: Name for the subscriber queue
            event_types: Optional filter for specific event types
            
        Returns:
            Dictionary with queue URL and subscription ARN
        """
        # Create SQS queue for this subscriber
        queue_name = f"{self.name}-{subscriber_name}-queue"
        response = self.sqs.create_queue(QueueName=queue_name)
        queue_url = response['QueueUrl']
        
        # Get queue ARN
        attrs = self.sqs.get_queue_attributes(
            QueueUrl=queue_url,
            AttributeNames=['QueueArn']
        )
        queue_arn = attrs['Attributes']['QueueArn']
        
        # Set queue policy to allow SNS
        policy = {
            "Version": "2012-10-17",
            "Statement": [{
                "Effect": "Allow",
                "Principal": {"Service": "sns.amazonaws.com"},
                "Action": "sqs:SendMessage",
                "Resource": queue_arn,
                "Condition": {
                    "ArnEquals": {"aws:SourceArn": self.topic_arn}
                }
            }]
        }
        
        self.sqs.set_queue_attributes(
            QueueUrl=queue_url,
            Attributes={'Policy': json.dumps(policy)}
        )
        
        # Subscribe queue to topic
        subscribe_params = {
            'TopicArn': self.topic_arn,
            'Protocol': 'sqs',
            'Endpoint': queue_arn
        }
        
        # Add filter policy if event types specified
        if event_types:
            filter_policy = {'event_type': event_types}
            subscribe_params['Attributes'] = {
                'FilterPolicy': json.dumps(filter_policy)
            }
        
        response = self.sns.subscribe(**subscribe_params)
        
        return {
            'queue_url': queue_url,
            'queue_arn': queue_arn,
            'subscription_arn': response['SubscriptionArn']
        }
    
    def receive_events(
        self,
        queue_url: str,
        max_events: int = 10
    ) -> List[Dict[str, Any]]:
        """
        Receive events from a subscriber queue.
        
        Args:
            queue_url: SQS queue URL
            max_events: Maximum events to receive
            
        Returns:
            List of events
        """
        response = self.sqs.receive_message(
            QueueUrl=queue_url,
            MaxNumberOfMessages=min(max_events, 10),
            WaitTimeSeconds=5
        )
        
        events = []
        for message in response.get('Messages', []):
            # SNS wraps the message
            sns_message = json.loads(message['Body'])
            event = json.loads(sns_message['Message'])
            event['receipt_handle'] = message['ReceiptHandle']
            events.append(event)
        
        return events
    
    def acknowledge_event(self, queue_url: str, receipt_handle: str):
        """Acknowledge (delete) a processed event."""
        self.sqs.delete_message(
            QueueUrl=queue_url,
            ReceiptHandle=receipt_handle
        )

# Example usage
print("Event Bus Example:")
print("-" * 40)
print("""
# Create event bus
bus = EventBus('orders')

# Create subscribers
email_sub = bus.subscribe('email-service', ['order.created', 'order.shipped'])
inventory_sub = bus.subscribe('inventory-service', ['order.created'])
analytics_sub = bus.subscribe('analytics-service')  # All events

# Publish events
bus.publish('order.created', {
    'order_id': '12345',
    'customer': 'alice@example.com',
    'items': [{'sku': 'ABC123', 'qty': 2}]
})

# Each subscriber receives the event in their queue
events = bus.receive_events(email_sub['queue_url'])
for event in events:
    print(f"Email service received: {event['event_type']}")
    # Process and acknowledge
    bus.acknowledge_event(email_sub['queue_url'], event['receipt_handle'])
""")
```
</details>

### Exercise 4: Lambda Function Wrapper

Create a wrapper class for invoking Lambda functions with retries and logging.

In [ ]:
# Exercise 4: Your code here

class LambdaInvoker:
    """Wrapper for Lambda invocations with retry logic."""
    
    def __init__(self, function_name: str):
        pass
    
    def invoke(
        self,
        payload: dict,
        retries: int = 3,
        async_invoke: bool = False
    ) -> dict:
        """Invoke the function with retry logic."""
        pass

<details>
<summary>Click to see solution</summary>

```python
import boto3
import json
import time
import logging
from typing import Dict, Any, Optional
from botocore.exceptions import ClientError

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class LambdaInvoker:
    """
    Wrapper for Lambda invocations with:
    - Automatic retries with exponential backoff
    - Logging
    - Error handling
    """
    
    def __init__(
        self,
        function_name: str,
        region: str = 'us-east-1'
    ):
        """
        Initialize invoker.
        
        Args:
            function_name: Lambda function name or ARN
            region: AWS region
        """
        self.function_name = function_name
        self.lambda_client = boto3.client('lambda', region_name=region)
        self._invocation_count = 0
        self._error_count = 0
    
    def invoke(
        self,
        payload: dict,
        retries: int = 3,
        async_invoke: bool = False,
        timeout: int = None
    ) -> Dict[str, Any]:
        """
        Invoke the Lambda function with retry logic.
        
        Args:
            payload: Input data for the function
            retries: Number of retry attempts
            async_invoke: If True, invoke asynchronously
            timeout: Optional timeout override
            
        Returns:
            Dictionary with invocation results
        """
        invocation_type = 'Event' if async_invoke else 'RequestResponse'
        
        for attempt in range(retries + 1):
            try:
                self._invocation_count += 1
                
                logger.info(
                    f"Invoking {self.function_name} "
                    f"(attempt {attempt + 1}/{retries + 1})"
                )
                
                start_time = time.time()
                
                response = self.lambda_client.invoke(
                    FunctionName=self.function_name,
                    InvocationType=invocation_type,
                    Payload=json.dumps(payload)
                )
                
                duration = time.time() - start_time
                
                result = {
                    'success': True,
                    'status_code': response['StatusCode'],
                    'duration_ms': int(duration * 1000),
                    'attempt': attempt + 1,
                    'function_error': response.get('FunctionError'),
                    'executed_version': response.get('ExecutedVersion')
                }
                
                # Read payload for sync invocations
                if not async_invoke:
                    payload_data = response['Payload'].read().decode('utf-8')
                    if payload_data:
                        result['payload'] = json.loads(payload_data)
                    
                    # Check for function errors
                    if response.get('FunctionError'):
                        self._error_count += 1
                        result['success'] = False
                        logger.error(
                            f"Function error: {result.get('payload', {}).get('errorMessage')}"
                        )
                
                logger.info(
                    f"Invocation completed in {result['duration_ms']}ms "
                    f"(status: {result['status_code']})"
                )
                
                return result
                
            except ClientError as e:
                self._error_count += 1
                error_code = e.response['Error']['Code']
                
                # Determine if retryable
                retryable_errors = [
                    'ServiceException',
                    'TooManyRequestsException',
                    'EC2ThrottledException'
                ]
                
                if error_code in retryable_errors and attempt < retries:
                    # Exponential backoff
                    wait_time = (2 ** attempt) + (time.time() % 1)
                    logger.warning(
                        f"Retryable error {error_code}, "
                        f"waiting {wait_time:.1f}s before retry"
                    )
                    time.sleep(wait_time)
                    continue
                
                logger.error(f"Lambda invocation failed: {e}")
                return {
                    'success': False,
                    'error_code': error_code,
                    'error_message': e.response['Error']['Message'],
                    'attempt': attempt + 1
                }
            
            except Exception as e:
                self._error_count += 1
                logger.error(f"Unexpected error: {e}")
                return {
                    'success': False,
                    'error': str(e),
                    'attempt': attempt + 1
                }
        
        return {
            'success': False,
            'error': 'Max retries exceeded'
        }
    
    def get_stats(self) -> Dict[str, int]:
        """Get invocation statistics."""
        return {
            'total_invocations': self._invocation_count,
            'total_errors': self._error_count,
            'success_rate': (
                (self._invocation_count - self._error_count) / 
                self._invocation_count * 100
                if self._invocation_count > 0 else 0
            )
        }

# Example usage
print("Lambda Invoker Example:")
print("-" * 40)
print("""
invoker = LambdaInvoker('my-function')

# Synchronous invocation with retries
result = invoker.invoke(
    payload={'name': 'Alice', 'action': 'greet'},
    retries=3
)

if result['success']:
    print(f"Response: {result['payload']}")
    print(f"Duration: {result['duration_ms']}ms")
else:
    print(f"Error: {result.get('error_message')}")

# Async invocation (fire and forget)
result = invoker.invoke(
    payload={'task': 'background_job'},
    async_invoke=True
)

# Get statistics
stats = invoker.get_stats()
print(f"Success rate: {stats['success_rate']:.1f}%")
""")
```
</details>

---

## Summary

In this notebook, we covered:

1. **DynamoDB**
   - Table creation with primary keys
   - CRUD operations (Create, Read, Update, Delete)
   - Queries and scans
   - Batch operations

2. **AWS Lambda**
   - Invoking functions synchronously and asynchronously
   - Handling responses and errors

3. **Amazon SQS**
   - Sending and receiving messages
   - Message visibility and deletion
   - Standard vs FIFO queues

4. **Amazon SNS**
   - Publishing notifications
   - Subscribing endpoints
   - Fan-out pattern

### Key Takeaways

- **DynamoDB**: Use queries over scans for efficiency
- **Lambda**: Choose sync/async based on use case
- **SQS**: Always delete messages after processing
- **SNS**: Great for broadcasting to multiple subscribers

### AWS Cost Warning

> **Pricing Considerations**:
> - **DynamoDB**: Pay for storage and read/write capacity
> - **Lambda**: Free tier includes 1M requests/month
> - **SQS**: First 1M requests/month free
> - **SNS**: First 1M requests/month free
>
> Always clean up unused resources!

---

## Next Steps

Continue to [04_practical_patterns.ipynb](04_practical_patterns.ipynb) to learn about:
- Error handling with botocore exceptions
- Pagination patterns
- Async operations
- Best practices for production